In [ ]:
import whisper

# Load Whisper model
model = whisper.load_model("base")

# Transcribe audio
result = model.transcribe("dream_audio.mp3")  # Replace with uploaded file path

# Transcribed text
dream_text = result['text']
print("Transcribed Dream:", dream_text)

In [ ]:
import openai

openai.api_key = "YOUR_OPENAI_API_KEY"

def clean_transcription(text):
    prompt = f"Please clean up this dream transcription and make it more readable: {text}"
    response = openai.Completion.create(
        engine="gpt-4",  # Or gpt-3.5-turbo
        prompt=prompt,
        max_tokens=500
    )
    return response.choices[0].text.strip()

cleaned_dream = clean_transcription(dream_text)
print("Cleaned Dream:", cleaned_dream)

In [ ]:
def generate_title(text):
    prompt = f"Generate a creative and fitting title for this dream: {text}"
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=prompt,
        max_tokens=20  # Limit title length
    )
    return response.choices[0].text.strip()

dream_title = generate_title(cleaned_dream)
print("Dream Title:", dream_title)

In [ ]:
from datetime import datetime
import json

# Get current date
date = datetime.now().strftime("%Y-%m-%d")

# Create a dream entry dictionary
dream_entry = {
    "date": date,
    "title": dream_title,
    "content": cleaned_dream
}

# Save to a JSON file (or append to an existing journal)
try:
    with open("dream_journal.json", "r") as f:
        dream_journal = json.load(f)
except FileNotFoundError:
    dream_journal = []

dream_journal.append(dream_entry)

with open("dream_journal.json", "w") as f:
    json.dump(dream_journal, f, indent=4)

print("Dream saved successfully!")

In [16]:
!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, Audio
from google.colab import output
import numpy as np
import io
import base64

# Create start and stop buttons
start_button = widgets.Button(description="Start Recording")
stop_button = widgets.Button(description="Stop Recording")

recording = False

# Function to start recording
def start_recording(b):
    global recording
    recording = True
    output.eval_js('startRecording()')
    print("Recording started...")

# Function to stop recording and save the file
def stop_recording(b):
    global recording
    if not recording:
        print("No recording in progress.")
        return

    recording = False
    audio_data = output.eval_js('stopRecording()')

    if audio_data is None:
        print("No audio data recorded.")
        return

    # Decode base64 data
    audio_bytes = base64.b64decode(audio_data['audio'])

    # Create and display Audio widget
    audio_obj = Audio(data=audio_bytes, rate=audio_data['sampleRate'])
    display(audio_obj)

    # Save as WAV file
    with open("recorded_audio.wav", "wb") as f:
        f.write(audio_bytes)

    print("Audio saved as 'recorded_audio.wav'")

In [17]:
# Connect buttons to functions
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)

# Display buttons
display(start_button, stop_button)

# JavaScript to handle audio recording
output.eval_js('''
    let recorder, stream;

    async function startRecording() {
        stream = await navigator.mediaDevices.getUserMedia({ audio: true });
        recorder = new MediaRecorder(stream);
        chunks = [];
        recorder.ondataavailable = e => chunks.push(e.data);
        recorder.start();
    }

    function stopRecording() {
        return new Promise(resolve => {
            recorder.onstop = () => {
                const blob = new Blob(chunks, { type: 'audio/wav' });
                chunks = [];
                stream.getTracks().forEach(track => track.stop());
                const reader = new FileReader();
                reader.onload = () => {
                    const base64data = reader.result.split(',')[1];
                    resolve({audio: base64data, sampleRate: 44100});
                };
                reader.readAsDataURL(blob);
            };
            recorder.stop();
        });
    }
''')

Button(description='Start Recording', style=ButtonStyle())

Button(description='Stop Recording', style=ButtonStyle())

Recording started...


Audio saved as 'recorded_audio.wav'


In [10]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [9]:
from google.colab import output
output.disable_custom_widget_manager()